# Notebook Goal:
Demonstrate HMM classification and use a neural network to predict chrods

In [4]:
cd ../

/Users/mattgraziano/Google Drive/Documents/Learning/IU/Fall18/engr511/project/engr511


In [1]:
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
import numpy as np
import librosa
from livelossplot import PlotLossesKeras
import scipy.fftpack

Using TensorFlow backend.


In [2]:
from keras.models import Model
from keras.layers import Input, Dense, Softmax, Dropout, BatchNormalization, Activation

from keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder

import keras

In [5]:
from src.Progression_Modeling import Load, Prepare, Mel_transform, normalize, melspectrogram, trim, CQT_transform, pseudo_cqt, Chroma
import soundfile as sf
from scipy.signal import stft

In [10]:
def get_spectrogram(signal, sr=44100, spectype="mel"):
    if spectype == "mel":
        spectrogram = melspectrogram(Prepare(signal), sr=sr, fmin=C2, fmax=C7, n_fft=2048 * 2, hop_length=1024)
        spectrogram_norm = pow(normalize(spectrogram), .25)
        
    elif spectype == "cqt":
        A1 = 55.00
        spectrogram = abs(pseudo_cqt(Prepare(signal), sr, fmin=A1, n_bins=120, bins_per_octave=24, sparsity=.95, window=('kaiser', 10)))
        spectrogram_norm = normalize(spectrogram, norm='l1', axis=0)
        
    elif spectype == "chroma":
        CQT = CQT_transform(Prepare(signal), sr)
        spectrogram = Chroma(CQT)
        spectrogram_norm = normalize(spectrogram, norm='l1', axis=0)
        
    elif spectype == "stft":
        spectrogram = stft(signal, fs=sr, window="hann")[2]
        spectrogram_norm = np.real(normalize(spectrogram, norm='l1', axis=0))

    return spectrogram_norm

In [11]:
def get_clean_x_avg_all(signal, sr=44100, spectype="mel"):
    spectrogram = get_spectrogram(signal, sr=sr, spectype=spectype)

    # Get column averages of the max n columns in lieu of using the entire spectrogram
    clean = np.mean(spectrogram, axis=1)
    
    return clean

In [12]:
recording_filenames = glob("data/recordings_new/*/*/*.wav", recursive=True)

In [ ]:
progression_filenames = glob("data/recordings_new/")

## Import progressions

In [16]:
from src.Progression_Modeling import HMM_Stack_Transform

In [25]:
# import progressions
recorded_progression_filenames = glob("data/Progressions/*/*.wav")

In [26]:
recorded_progression_filenames[0]

'data/Progressions/Inversions/Min_Maj_Maj7_Maj_Min_END_1.wav'

In [29]:
# get prog y
# NOTE: We are parsing folder structure and filenames
y_progressions = [prod.split("/")[3].split("_END")[0] for prod in recorded_progression_filenames]

In [30]:
y_progressions

['Min_Maj_Maj7_Maj_Min',
 'Min_Maj_Maj_Maj_Min',
 'Maj_Maj_Maj_Dim_Maj',
 'Maj_Maj_Maj_Maj_Maj',
 'Min_Maj_Maj7_Maj_Min',
 'Min_Maj_Maj7_Min_Min',
 'Maj_Maj7_Dim_Min_Maj',
 'Maj_Maj_Maj7_Min_Maj',
 'Maj_Maj_Maj_Maj_Maj',
 'Maj7_Maj_Min_Maj_Maj7',
 'Min_Maj_Min_Min_Min',
 'Maj7_Maj_dim_Maj_Maj7',
 'Min_Min_Maj_Dim_Min',
 'Min_Min_Maj7_Maj_Min',
 'Maj7_Maj_Maj_Min_Maj7',
 'Maj_Maj_Dim_Maj_Maj',
 'Maj_Maj_Min_Dim_Maj',
 'Maj_Dim_Min_Maj_Maj']

In [32]:
y_progressions_vals = [item.split("_") for item in y_progressions]

In [33]:
y_progressions_vals

[['Min', 'Maj', 'Maj7', 'Maj', 'Min'],
 ['Min', 'Maj', 'Maj', 'Maj', 'Min'],
 ['Maj', 'Maj', 'Maj', 'Dim', 'Maj'],
 ['Maj', 'Maj', 'Maj', 'Maj', 'Maj'],
 ['Min', 'Maj', 'Maj7', 'Maj', 'Min'],
 ['Min', 'Maj', 'Maj7', 'Min', 'Min'],
 ['Maj', 'Maj7', 'Dim', 'Min', 'Maj'],
 ['Maj', 'Maj', 'Maj7', 'Min', 'Maj'],
 ['Maj', 'Maj', 'Maj', 'Maj', 'Maj'],
 ['Maj7', 'Maj', 'Min', 'Maj', 'Maj7'],
 ['Min', 'Maj', 'Min', 'Min', 'Min'],
 ['Maj7', 'Maj', 'dim', 'Maj', 'Maj7'],
 ['Min', 'Min', 'Maj', 'Dim', 'Min'],
 ['Min', 'Min', 'Maj7', 'Maj', 'Min'],
 ['Maj7', 'Maj', 'Maj', 'Min', 'Maj7'],
 ['Maj', 'Maj', 'Dim', 'Maj', 'Maj'],
 ['Maj', 'Maj', 'Min', 'Dim', 'Maj'],
 ['Maj', 'Dim', 'Min', 'Maj', 'Maj']]

In [19]:
recored_progression_filenames

['data/Progressions/Inversions/Min_Maj_Maj7_Maj_Min_END_1.wav',
 'data/Progressions/Inversions/Min_Maj_Maj_Maj_Min_END_1.wav',
 'data/Progressions/Inversions/Maj_Maj_Maj_Dim_Maj_END_1.wav',
 'data/Progressions/Inversions/Maj_Maj_Maj_Maj_Maj_END_1.wav',
 'data/Progressions/Different Instrument/Min_Maj_Maj7_Maj_Min_END_2.wav',
 'data/Progressions/Different Instrument/Min_Maj_Maj7_Min_Min_END_1.wav',
 'data/Progressions/Different Instrument/Maj_Maj7_Dim_Min_Maj_END_1.wav',
 'data/Progressions/Different Instrument/Maj_Maj_Maj7_Min_Maj_END_1.wav',
 'data/Progressions/Different Instrument/Maj_Maj_Maj_Maj_Maj_END_2.wav',
 'data/Progressions/Different Instrument/Maj7_Maj_Min_Maj_Maj7_END_1.wav',
 'data/Progressions/Regular/Min_Maj_Min_Min_Min_END_1.wav',
 'data/Progressions/Regular/Maj7_Maj_dim_Maj_Maj7_END_1.wav',
 'data/Progressions/Regular/Min_Min_Maj_Dim_Min_END_1.wav',
 'data/Progressions/Regular/Min_Min_Maj7_Maj_Min_END_1.wav',
 'data/Progressions/Regular/Maj7_Maj_Maj_Min_Maj7_END_1.wav'

In [15]:
transform = "chroma"

raw_signals = []
samp_rates = []
x_train = []
for filename in tqdm(recording_filenames):
    sound, sr = sf.read(filename)

    raw_signals.append(sound)
    samp_rates.append(sr)
    cleaned = get_clean_x_avg_all(sound, sr=sr, spectype=transform)
    x_train.append(cleaned.flatten())

# Standardize input
x_train = np.array(x_train)
x_train = (x_train - x_train.min(0)) / x_train.ptp(0)


# Get y categories
y_train = [i.split('/')[2] for i in recording_filenames]
le_chord_structure = LabelEncoder()
le_chord_structure.fit(y_train)
y_train_int = to_categorical(le_chord_structure.transform(y_train))


KeyboardInterrupt: 